<a href="https://colab.research.google.com/github/komo135/tradingrl/blob/master/dqn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google ドライブをマウントするには、このセルを実行してください。
from google.colab import drive
drive.mount('/content/drive')

%cd drive/My Drive
%load_ext Cython

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [2]:
!pip install ta

In [0]:
import numpy as np
from scipy.special import expit
import pandas as pd
import pickle
from numba import jit as njit
from functools import lru_cache
from multiprocessing import Pool
import os
import time
import random
import ta
from net import *
from memory import *
from reward import *
import traceback
from IPython.display import clear_output

In [0]:
def swish(x):
    x *= tf.nn.sigmoid(x)
    return x

def actor(x):
  with tf.variable_scope("actor", reuse=False):
    with tf.variable_scope("main", reuse=False):
      x1 = tf.keras.layers.Conv1D(128,8,1,padding="causal",activation=tf.nn.relu)(x)
      x2 = tf.keras.layers.Conv1D(128,4,1,padding="causal",activation=tf.nn.relu)(x)
      x3 = tf.keras.layers.Conv1D(128,2,1,padding="causal",activation=tf.nn.relu)(x)
      x = tf.keras.layers.Add()([x1,x2,x3])
      x = tf.keras.layers.Conv1D(128,2,1,padding="causal",activation=tf.nn.relu)(x)
      f = tf.keras.layers.Flatten()(x)
    with tf.variable_scope("sub", reuse=False):
      out = tf.keras.layers.Dense(128,tf.nn.relu)(f)
      out = tf.keras.layers.Dense(3,tf.tanh)(out)
    with tf.variable_scope("main2", reuse=False):
      x = tf.keras.layers.Dense(128,tf.nn.relu)(f)
      x = tf.keras.layers.Concatenate()([out,x])
      x = tf.keras.layers.Dense(128,tf.nn.relu)(x)
      x = tf.keras.layers.Dense(3)(x)
      X = tf.keras.layers.Maximum()([x[:,0],x[:,1],x[:,2]])
  return X,x,out

In [0]:
class Agent:
    def __init__(self,path,window_siz):
      self.path = path
      self.window_size = window_size
      self.preproc()
      self.state_size = (None,self.window_size,self.df.shape[-1])
      print(self.state_size)
      self.memory = Memory(5000)
      self.rewards = reward2
      self.sess = tf.Session()
      self.STEP_SIZE = 24 * 5

      with tf.variable_scope("input"):
        self.state = tf.placeholder(tf.float32, self.state_size)
        self.new_state = tf.placeholder(tf.float32, self.state_size)
        self.action = tf.placeholder(tf.float32,(None,3))
        self.reward = tf.placeholder(tf.float32,(None,3))

      with tf.variable_scope("model", reuse=False):
        self.policy,self.q,self.out = actor(self.state)
      
      with tf.variable_scope("target", reuse=False):
        self.target_q,_,_ = actor(self.new_state)

      with tf.variable_scope("loss"):
        self.loss = loss = (0.5 * tf.reduce_mean((self.reward - self.q) ** 2))
        self.ploss = policy_loss = -tf.reduce_mean(self.policy)
        self.absolute_errors = tf.abs(self.reward - self.q)

      self.policy_sub_opt = tf.train.AdamOptimizer(1e-3).minimize(policy_loss, var_list=get_vars('model/actor/sub'))
      self.policy_main_opt = tf.train.AdamOptimizer(1e-3).minimize(loss, var_list=get_vars('model/actor/main')+get_vars('model/actor/main2'))

      self.target_update = tf.group([tf.assign(v_targ, 0.995*v_targ + (1-0.995)*v_main)
                                for v_main, v_targ in zip(get_vars('model'), get_vars('target'))])

      target_init = tf.group([tf.assign(v_targ, v_main)
                              for v_main, v_targ in zip(get_vars('model'), get_vars('target'))])

      self.sess.run(tf.global_variables_initializer())
      # self.sess.run(target_init)
    def preproc(self):
        self.dat = df = pd.read_csv(self.path)
        s = np.asanyarray(ta.stoch(df["High"],df["Low"],df["Close"],14)).reshape((-1, 1)) - np.asanyarray(ta.stoch_signal(df["High"],df["Low"],df["Close"],14)).reshape((-1, 1))
        x = np.asanyarray(ta.daily_return(df["Close"])).reshape((-1,1))
        m = np.asanyarray(ta.macd_diff(df["Close"])).reshape((-1,1))
        cross1 = np.asanyarray(ta.ema(self.dat["Close"],12)).reshape((-1, 1)) - np.asanyarray(ta.ema(self.dat["Close"],5)).reshape((-1, 1))
        trend = np.asanyarray(df[["Close"]]) - np.asanyarray(ta.ema(self.dat["Close"],50)).reshape((-1, 1))
        # x = s
        x = np.concatenate([m,s,x,cross1,trend], 1)
        y = np.asanyarray(self.dat[["Open"]])

        gen = tf.keras.preprocessing.sequence.TimeseriesGenerator(x, y, self.window_size)
        self.x = []
        self.y = []
        for i in gen:
            self.x.extend(i[0].tolist())
            self.y.extend(i[1].tolist())
        self.x = np.asanyarray(self.x)[100:] #.reshape((-1, self.window_size, x.shape[-1]))
        self.y = np.asanyarray(self.y)[100:]

        self.df = self.x[100:]
        self.trend = self.y[100:]

    def _construct_memories_and_train(self,i, replay=None):
      try:
          tree_idx, replay = self.memory.sample(128)
      except:
          self.memory = Memory(5000)

      states = np.array([a[0][0] for a in replay])
      new_states = np.array([a[0][3] for a in replay])
      actions = np.array([a[0][1] for a in replay]).reshape((-1, 3))
      rewards = np.array([a[0][2] for a in replay]).reshape((-1, 1))

      target_q = self.sess.run(self.target_q, feed_dict={self.new_state:new_states}).reshape((-1,1))
      q = self.sess.run(self.q,feed_dict={self.state:states}).reshape((-1,3))

      for I in range(128):
        q[I,np.argmax(actions[I])] = rewards[I] + 0.99 * target_q[I]

      step_ops = [self.absolute_errors,self.policy_main_opt]
      for _ in range(1):
        absolute_errors,_ = self.sess.run(step_ops, feed_dict={self.state: states, self.new_state: new_states, self.reward: q})
      if i > 10:
        if (i+1) % 2 == 0:
              loss,_ = self.sess.run([self.loss,self.policy_sub_opt], feed_dict={self.state: states,self.reward: q})
              # print(loss)
          # print([rewards,loss])
      self.sess.run(self.target_update)
      ae = []
      # self.memory.batch_update(tree_idx, ae)

    def _select_action(self, state, i):
        # self.policy_out
        prediction,q = self.sess.run([self.out,self.q], feed_dict={self.state: [state]})
        prediction,q = prediction[0],q[0]
        prediction *= q

        noise = 0. if (i + 1) % 5 == 0 else 0.2
        if noise != 0.:
            prediction = np.clip(prediction, -1, 1)
            prediction += noise * np.random.randn(3)
            prediction = np.clip(prediction, -1, 1)
        action = np.argmax(prediction)
        self.pred = prediction

        return action

    def prob(self,history):
        prob = np.asanyarray(history)
        a = np.mean(prob == 0)
        b = np.mean(prob == 1)
        c = 1 - (a + b)
        prob = [a,b,c]
        return prob

    def discount_rewards(self, r, running_add):
        running_add = running_add * 0.99 + r
        return running_add
        
    def nstep(self,r):
        running_add = 0.0
        for t in range(len(r)):
            running_add += 0.99 * r[t]

        return running_add

    def run(self, spread=10, pip_cost=1000, los_cut=300, day_pip=24, n=10,step=100):
        spread = spread / pip_cost
        self.rand = np.random.RandomState()
        lc = los_cut / pip_cost
        for i in range(100000):
            # if (i - 1) % step == 0:
            h = self.rand.randint(self.x.shape[0]-(self.STEP_SIZE+1))
            self.df = self.x[h:h+self.STEP_SIZE]
            self.trend = self.y[h:h+self.STEP_SIZE]
            done = 0.0
            position = 3
            pip = []
            provisional_pip = []
            running_add = 0.0
            total_pip = 0.0
            old_reword = 0.0
            states = []
            h_a = []
            h_r = []
            h_p = []
            old = np.asanyarray(0)
            self.history = []
            for t in  range(0, len(self.trend)-1):
                action = self._select_action(self.df[t],i)
                h_a.append(self.pred)
                self.history.append(action)
                
                states,provisional_pip,position,total_pip = self.rewards(self.trend[t],pip,provisional_pip,
                                    action,position,states,pip_cost,spread,total_pip,lc=los_cut/2/pip_cost)
                h_p.append(position)
                reward =  (total_pip - old_reword) * 100
                old_reword = total_pip
                h_r.append(reward)

                # running_add = self.discount_rewards(reward,running_add)
                # r_d = int(running_add * (pip_cost / 100)) * 100
                # r_d = int(np.mean(np.array(provisional_pip) > 0)*100) if len(provisional_pip) != 0 else 0.
                # if t == len(self.trend)-1:
                #     done = 1.0
                # exp = self.df[t], h_a[t], r_d, self.df[t+1], done
                # self.memory.store(exp)
            for t in range(0, len(self.trend)-1):
                tau = t - n + 1
                if tau >= 0:
                    rewards = self.nstep(h_r[tau+1:tau+n])
                    exp = self.df[t], h_a[t], int(rewards), self.df[t+1], done
                    self.memory.store(exp)

            try:
              self._construct_memories_and_train(i)
            except:
              traceback.print_exc()

            if i % 1000 == 0:
              clear_output()

            if (i + 1) % 5 == 0:
                # loss = np.mean(ae)
                self.pip = np.asanyarray(provisional_pip) * pip_cost
                self.pip = [p if p >= -los_cut else -los_cut for p in self.pip]
                self.total_pip = np.sum(self.pip)
                mean_pip = self.total_pip / (t + 1)
                trade_accuracy = np.mean(np.asanyarray(self.pip) > 0)
                self.trade = trade_accuracy
                mean_pip *= day_pip
                prob = self.prob(self.history)
                position_prob = self.prob(h_p)

                # print("loss =", loss)
                # print("")
                print('action probability = ', prob)
                print("buy = ", position_prob[1], " sell = ", position_prob[-1])
                print('trade accuracy = ', trade_accuracy)
                print('epoch: %d, total rewards: %f, mean rewards: %f' % (i + 1, float(self.total_pip), float(mean_pip)))
                print("")

In [0]:
window_size = 30
path = "audpred60.csv"
agent = Agent(path,window_size)

agent.run()

action probability =  [0.3025210084033613, 0.2857142857142857, 0.41176470588235303]
buy =  0.5966386554621849  sell =  0.40336134453781514
trade accuracy =  0.44285714285714284
epoch: 1005, total rewards: 1328.000000, mean rewards: 267.831933

action probability =  [0.08403361344537816, 0.7058823529411765, 0.2100840336134453]
buy =  0.1092436974789916  sell =  0.8907563025210083
trade accuracy =  0.43010752688172044
epoch: 1010, total rewards: 28445.000000, mean rewards: 5736.806723

action probability =  [0.13445378151260504, 0.3949579831932773, 0.47058823529411764]
buy =  0.2857142857142857  sell =  0.7142857142857143
trade accuracy =  0.31746031746031744
epoch: 1015, total rewards: -4871.000000, mean rewards: -982.386555



In [0]:
agent.absolute_errors